In [ ]:
"""
    Radar Jupyter Notebook

    You can use this notebook as a way to get started with your first template writing, without setting up docker and the infrastructure
    - Open in VSCode
    - Play with the "code" variable which simulates a radar rule execution
    - Default configruation points to tests/mocks/ast_mock.json as the ast source
"""

import json

show_debug = True  # Switch to True to see debug output in the template results


def parse_radar_rule(code: str, yaml_data) -> list:
    from pathlib import Path
    import sys

    sys.path.append(str(Path("api").absolute()))
    from api.utils.dsl.dsl import (
        inject_code_lines,
        wrapped_exec,
        process_template_outputs,
    )

    with open(Path("api/tests/mocks/ast_mock.json"), "r") as file:
        generated_ast = json.load(file)
    modified_code = inject_code_lines(code, [f"ast = {generated_ast}"])
    template_outputs = wrapped_exec(modified_code)
    results = process_template_outputs(template_outputs, yaml_data)
    if not show_debug and results.get("debug") is not None:
        del results["debug"]
    return results


###############################################
# Simulate a rule here                        #
###############################################
code = """
for source, nodes in ast["sources"].items():
    invoke_nodes, cpi_nodes = find_cpi_nodes(nodes)
    transfer_call = find_nodes_by_order(cpi_nodes, ["spl_token", "instruction", "transfer"])
    if not transfer_call:
        continue
    if find_compare_operations_between(nodes, "spl_token", "token_program"):
        continue
    for invoke_node in invoke_nodes:
        print(invoke_node)
        print("dam", invoke_node)
"""

yaml_data = {
    "name": "Debug Template",
    "severity": "Medium",
    "certainty": "Medium",
    "description": "Debug template description",
}

results = parse_radar_rule(code, yaml_data)
if results.get("locations") is not None or results.get("debug") is not None:
    print(json.dumps(results, indent=4).replace('\\"', ''))